## Important Libbraries

In [ ]:
!pip install -U datasets huggingface_hub transformers[torch] evaluate peft --quiet

In [ ]:
from transformers import TrainingArguments, Trainer

## Pre-Processing and Loading

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from datasets import load_dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("nyu-mll/multi_nli")

In [ ]:
train_len = len(dataset['train'])

In [ ]:
dataset['train'] = dataset['train'].shuffle(seed=42).filter(lambda example, idx: idx < train_len * 0.25, with_indices=True)

In [ ]:
dataset

In [ ]:
dataset['train'][0]

In [ ]:
model = RobertaForSequenceClassification.from_pretrained('FacebookAI/roberta-large', num_labels=3)
tokenizer = RobertaTokenizer.from_pretrained('FacebookAI/roberta-large')

In [ ]:
def tokenize_func(examples):
    return tokenizer(examples['premise'], examples['hypothesis'], truncation=True)

tokenized_dataset = dataset.map(tokenize_func, batched=True)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

## Training Process

In [ ]:
import torch

In [ ]:
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
training_args = TrainingArguments(
    output_dir = './result',
    save_steps=0.5,
    learning_rate=2e-5,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    warmup_steps=0.1,
    weight_decay=0.01,
    report_to='none',
    evaluation_strategy="steps",
    eval_steps=1/4,
    overwrite_output_dir=True,
    fp16=True,
)


trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['validation_matched'],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.evaluate(eval_dataset=tokenized_dataset['validation_mismatched'])